<h3>Import libraries & packages</h3>

In [76]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, Lambda
from keras.layers import Conv2D, Activation, AveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K


<h3>Load data</h3>

In [46]:
data = []
labels = []

PATH_DATA = "./SMILEsmileD/SMILEs/"

directory_positive = os.fsencode(PATH_DATA + "positives/positives7/")

for file in os.listdir(directory_positive):
    filename = os.fsdecode(file)
    img = cv2.cvtColor(cv2.imread(PATH_DATA + "positives/positives7/" + filename), \
                       cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (32, 32))
    img = img_to_array(img)
    data.append(img)
    
    label = "positive"
    labels.append(label)

directory_negative = os.fsencode(PATH_DATA + "negatives/negatives7/")

for file in os.listdir(directory_negative):
    filename = os.fsdecode(file)
    img = cv2.cvtColor(cv2.imread(PATH_DATA + "negatives/negatives7/" + filename), \
                       cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (32, 32))
    img = img_to_array(img)
    data.append(img)
    
    label = "negative"
    labels.append(label)

<h3>Data preprocessing</h3>

In [47]:
# Scale the raw pxel intensities to the range [0, 1]
data = np.array(data, dtype=np.float32) / 255.0
labels = np.array(labels)

# Convert the labels from strings to vectors
le = LabelEncoder().fit(labels)
labels = to_categorical(le.transform(labels), 2) # one hot vector

# Account for skew in the labbeled data
classTotals = labels.sum(axis=0) # total # of examples per class
classWeight = classTotals.max() / classTotals # scaling these totals

<h3>Train/test split</h3>

The stratify argument will make sure that the test split has the same proportion of positives and negatives samples than the train dataset.

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, \
                                                    test_size=0.2, \
                                                    stratify=labels, \
                                                    random_state=42)

<h3>Building model</h3>

In [89]:
def LeNet_model(input_shape=(32, 32, 1), num_classes=2):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape, name="Main_input")
    
    # 1st CONV LAYER : kernel (5x5), strides(1, 1), 6 filters, no pad
    X = Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), \
               kernel_initializer=glorot_uniform(seed=0), \
               padding="valid", \
               name="Conv_1")(X_input)
    # Add non linearity with Rectified Linear Unit (ReLU)
    X = Activation('tanh', name="Tanh_1")(X)
    # Average Pooling 
    X = AveragePooling2D(pool_size=(2, 2), name="Avg_Pool_1")(X)
    
    # 2nd CONV LAYER : kernel (5x5), strides (1, 1), 16 filters, no pad
    X = Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), \
               kernel_initializer=glorot_uniform(seed=0), \
               padding="valid", \
               name="Conv_2")(X)
    X = Activation('tanh', name="Tanh_2")(X)
    X = AveragePooling2D(pool_size=(2, 2), name="Avg_Pool_2")(X)
    
    # 3rd CONV LAYER : kernel (5x5), strides (1, 1), 16 filters, no pad
    X = Conv2D(filters=120, kernel_size=(5, 5), strides=(1, 1), \
               kernel_initializer=glorot_uniform(seed=0), \
               padding="valid", \
               name="Conv_3")(X)
    
    # Flatten
    X = Flatten(name="Flatten")(X)
    
    # Fully connected layer 1: 84 units
    X = Dense(84, name="FC_1")(X)
    X = Activation('tanh', name="Tanh_3")(X)
    
    # Output layer
    X = Dense(num_classes, name="Output")(X)

    # Custom activation function (Radial Basis Function - RBF)
    l2_norm = lambda a, b: K.sqrt(K.sum(((a - b) ** 2), axis=0, keepdims=True))
    def rbf(x, gamma=1.0):
         return K.exp(-1 * gamma * l2_norm(x[0], x[1]) ** 2)    
    
    X = Activation(rbf)(X)
    
    # Create model
    model = Model(inputs=X_input, outputs=X, name="LeNet ConvNet")
    
    
    
    return model

In [90]:
model = LeNet_model()

We can take a peek at the overall structure we just made by calling the method <i>summary()</i>:

In [91]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Main_input (InputLayer)      (None, 32, 32, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
Tanh_1 (Activation)          (None, 28, 28, 6)         0         
_________________________________________________________________
Avg_Pool_1 (AveragePooling2D (None, 14, 14, 6)         0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 10, 10, 16)        2416      
_________________________________________________________________
Tanh_2 (Activation)          (None, 10, 10, 16)        0         
_________________________________________________________________
Avg_Pool_2 (AveragePooling2D (None, 5, 5, 16)          0         
__________

<h3>Compile Model</h3>

In [92]:
adam = Adam(lr=1e-4)

model.compile(optimizer=adam, \
              loss='categorical_crossentropy', \
              metrics=['accuracy'])

ValueError: logits and labels must have the same shape ((1,) vs (?, ?))